In [1]:
import time
import numpy as np
import pyscf
import scipy as sp
from pyscf import gto, scf, cc
from pyscf.cc.ccd import CCD

def restricted_HF_convE_and_DM(mol, conv_tol = 1e-10):
    """Restricted Hartree-Fock method, following Szabo and Ostlund.
    """
    start = time.time()

    n_a = mol.nelec[0]

    T = mol.intor("int1e_kin")
    Vnuc = mol.intor("int1e_nuc")
    hcore = T + Vnuc
    S = mol.intor("int1e_ovlp")
    h2e = mol.intor('int2e', aosym="s1") # most expensive, least intelligent way of doing this.

    convE = 10
    convDM = 10
    convComm = 10

    # we now implement checking convergence of both density matrix as well as energy.

    s, U = np.linalg.eigh(S)
    
    s_inv_sqrt = np.diag([x**(-0.5) for x in s])

    X = U @ s_inv_sqrt @ U.conj().T

    F0 = X.T @ hcore @ X
    _, C0 = np.linalg.eigh(F0)
    C = X @ C0
    P = 2 * C[:, :n_a] @ C[:, :n_a].T

    Eprev = None

    while convE > conv_tol or convDM > conv_tol or convComm > conv_tol:
        
        G = np.einsum('ij,klji->kl',P,h2e) - 0.5 * np.einsum('ij,kijl->kl',P,h2e)
        F = hcore + G

        E0 = 0.5 * np.einsum('ij,ij->',P,hcore+F)

        if Eprev is not None:
            convE = np.abs(E0 - Eprev)

        convComm = np.linalg.norm(F @ P @ S - S @ P @ F, 'fro')

        if convE < conv_tol and convDM < conv_tol and convComm < conv_tol:
            break
        
        Eprev = E0

        Fp = X.conj().T @ F @ X
        eps, Cp = np.linalg.eigh(Fp)
        C = X @ Cp

        Pnew = 2 * C[:,0:n_a] @ C[:,0:n_a].conj().T
        convDM = np.linalg.norm(Pnew-P)
        P = Pnew

    E0 = E0 + mol.energy_nuc()

    end = time.time()
    print(f"Converged in {end - start} seconds")

    return E0, P, F, C


In [11]:
def rccsd_Koch(mol, rhf_result, norb, nelec):
    # Chemical Physics Letters 228 (1994) 233-238 

    nocc = nelec//2
    h1e = mol.intor("int1e_kin") + mol.intor("int1e_nuc")
    gpqrs = mol.intor('int2e', aosym="s1")

    C = rhf_result[3]

    h1e_mo = C.T @ h1e @ C
    gpqrs_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl',gpqrs,C,C,C,C)

    orbital_energies = np.diag(C.T @ rhf_result[2] @ C)

    def generateX(t1):
        return np.eye(norb) - t1
    
    def generateY(t1):
        return np.eye(norb) + t1.T

    t1ai = np.zeros((norb,norb))

    Lpqrs_mo = 2*gpqrs_mo - np.einsum('pqrs->psrq',gpqrs_mo)

    def d1e(a,i):
        return orbital_energies[a] - orbital_energies[i]
    
    def d2e(a,i,b,j):
        return orbital_energies[a] + orbital_energies[b] - orbital_energies[i] - orbital_energies[j]

    t2ijab = np.zeros((norb,norb,norb,norb))
    for i in range(nocc,norb):
        for j in range(nocc,norb):
            for a in range(nocc):
                for b in range(nocc):
                    t2ijab[i,j,a,b] = Lpqrs_mo[a,i,b,j]/d2e(a,i,b,j)
    
    converged = False

    ecorr_prev = None

    while not converged:
        x = generateX(t1ai)
        y = generateY(t1ai)

        h1e_mo_mod = np.einsum('pr,rs,qs->pq',x,h1e_mo,y)
        gpqrs_mo_mod = np.einsum('pt,rv,qu,sw,tuvw->pqrs',x, x, y, y, gpqrs_mo)
        Lpqrs_mo_mod = 2*gpqrs_mo_mod - np.einsum('pqrs->psrq',gpqrs_mo_mod)

        uijab = 2*t2ijab - np.einsum('ijab->ijba',t2ijab)

        Finac_mod = h1e_mo_mod + np.einsum('kkpq->pq',Lpqrs_mo_mod[:nocc,:nocc,:,:])

        omegaA_aibj = 2*np.einsum('klab,kilj->aibj', t2ijab, gpqrs_mo_mod + np.einsum('ijcd,kcld->kilj',t2ijab,gpqrs_mo))
        omegaB_aibj = 2*np.einsum('ijcd,acbd->aibj',t2ijab,gpqrs_mo_mod)
        omegaC_aibj = -0.5*np.einsum('jkcb,kiac->aibj',t2ijab,gpqrs_mo_mod - 0.5*np.einsum('liad,lckd->kiac',t2ijab,gpqrs_mo))-np.einsum('ikcb,kjac->aibj',t2ijab,gpqrs_mo_mod-0.5*np.einsum('ljad,lckd->kjac',t2ijab,gpqrs_mo))
        omegaD_aibj = 0.5*np.einsum('jkbc,aikc->aibj',uijab,Lpqrs_mo_mod + 0.5*np.einsum('ilad,ldkc->aikc',uijab,Lpqrs_mo))
        omegaE_aibj = np.einsum('ijac,bc->aibj',t2ijab,Finac_mod - np.einsum('lmdb,ldmc->bc',t2ijab,Lpqrs_mo))-np.einsum('ikab,kj->aibj',t2ijab,Finac_mod+np.einsum('jmde,mekd->kj',t2ijab,Lpqrs_mo))
        omegaF_aibj = 2*gpqrs_mo_mod

        omega2_aibj = omegaA_aibj + omegaB_aibj + 2*(omegaC_aibj+omegaD_aibj+omegaE_aibj) + 2*np.einsum('aibj->bjai',omegaC_aibj+omegaD_aibj+omegaE_aibj) + omegaF_aibj

        omegaG_ai = 2*np.einsum('ikcd,kdac->ai',t2ijab,Lpqrs_mo_mod)
        omegaH_ai = -2*np.einsum('klad,ldki->ai',t2ijab,Lpqrs_mo_mod)
        omegaI_ai = 2*np.einsum('ikac,kc->ai',uijab,Finac_mod)
        omegaJ_ai = 2*Finac_mod

        omega1_ai = omegaG_ai + omegaH_ai + omegaI_ai + omegaJ_ai
        
        ecorr = 0.5*(np.einsum('ijab,iajb->',t2ijab,Lpqrs_mo) + np.einsum('ai,bj,iajb->',t1ai,t1ai,Lpqrs_mo))

        print(ecorr)
        if ecorr_prev is not None:
            if np.abs(ecorr_prev - ecorr) < 1e-8:
                converged = True
        
        if not converged:
            for i in range(nocc):
                for j in range(nocc):
                    for a in range(nocc,norb):
                        for b in range(nocc,norb):
                            t2ijab[i,j,a,b] = t2ijab[i,j,a,b] - omega2_aibj[a,i,b,j]/(2*d2e(a,i,b,j))
            
            for i in range(nocc):
                for a in range(nocc,norb):
                    t1ai[a,i] = t1ai[a,i] - omega1_ai[a,i]/(2*d1e(a,i))
        
        ecorr_prev = ecorr

    print("Final correlation energy: ", ecorr)
    return ecorr, t1ai, t2ijab



In [12]:
## pyscf calculation

from pyscf import gto, scf, cc
from pyscf.cc.ccd import CCD

import numpy as np
import scipy as sp

mol = gto.Mole()
mol.atom = """
H    0.0    0.0    0.0
O    1.0    0.0    0.0
H    2.0    0.0    0.0
"""
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()

print("orbitals:",mol.nao)
print("elec:",mol.nelec)

soln = restricted_HF_convE_and_DM(mol)

mf = scf.RHF(mol)
start = time.time()
mf.kernel()
end = time.time()
print(mf.mo_energy)

print(start - end, "seconds for RHF")


start = time.time()
mycc = cc.CCSD(mf)
mycc.kernel()
end = time.time()
print(start - end, "seconds for CCSD")

mf.MP2().run()

orbitals: 7
elec: (5, 5)
Converged in 0.025646686553955078 seconds
converged SCF energy = -74.8415921601715
[-20.06419967  -1.1359358   -0.60909247  -0.28988608  -0.28988608
   0.40909646   0.90285811]
-0.5752365589141846 seconds for RHF
E(CCSD) = -74.8820371115352  E_corr = -0.04044495136374235
-0.4903275966644287 seconds for CCSD
E(MP2) = -74.8709226561204  E_corr = -0.0293304959489343
E(SCS-MP2) = -74.8746499179775  E_corr = -0.0330577578060407


In [14]:
from pyscf import gto, scf, cc
from pyscf.cc.ccd import CCD

import numpy as np
import scipy as sp

mol = gto.Mole()
mol.atom = """
H    0.0    0.0    0.0
O    1.0    0.0    0.0
H    2.0    0.0    0.0
"""
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()

soln = restricted_HF_convE_and_DM(mol)

rccsdsoln = rccsd_Koch(mol,soln,7,10)



Converged in 0.007720947265625 seconds
-0.018367081819837157
-0.03303232979428998
-0.03675304842407379
-0.03793767349475029
-0.038347404223603325
-0.03849625399699419
-0.03855242931627363
-0.03857434646198978
-0.038583158201262874
-0.03858679960703043
-0.03858834256162121
-0.038589011225129304
-0.03858930678857326
-0.038589439669463314
-0.03858950026587844
-0.038589528222863635
-0.03858954124260568
-0.038589547351114535
Final correlation energy:  -0.038589547351114535
